In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import numpy as np
import pandas as pd

#### Modeling assumptions:
- Model assuming negligible diffusion from border crossing
- Carriers are contagious from day 0

#### Epidemiologic assumptions:
Sources:
- Source https://annals.org/aim/fullarticle/2762808/incubation-period-coronavirus-disease-2019-covid-19-from-publicly-reported
- Source https://www.epicentro.iss.it/coronavirus/bollettino/Report-COVID-2019_17_marzo-v2.pdf
- Source https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm
- Source Vo Euganeo study (test perfomed in entire population)

#### Modeling:

**Discrete Markov Chain** modeling states:
- *Healthy* ($H$)
- *Carriers* ($C$)
- *Symptomatic* ($S$)
- *Hospitalizable* ($X$)
- *Dead* ($D$)
- *Immune* ($I$)


Possible transitions:
- $H \rightarrow C$: rate depends on transmission model (dobumented below)
- $C \rightarrow S$
- $C \rightarrow I$
- $S \rightarrow I$
- $S \rightarrow X$
- $H \rightarrow D$
- $H \rightarrow I$

$I$ and $D$ are absorbing states.

### Countermeasures
We use the virus $R_0$ to model the spreading of the disease. We assume are three scenarios for $R_0$: no countermeasures, mild countermeasures (social distancing) and strict countermeasures (total lockdown). We use $R_0=2.5$ (widely assumed to be the $R_0$ of COVID-19), $R_0=1.5$ and $R_0=0.9$ respectively. 


In [53]:
states = ['H', 'C', 'S', 'X', 'D', 'I']
state_index = dict(zip(states, [0, 1, 2, 3, 4, 5])) 
state_name = dict(zip(states, ["Healthy",
                               "Carrier",
                               "Symptomatic",
                               "Hospitalizable",
                               "Dead",
                               "Immune"]))

#these are three scenarios for R_0: no countermeasures, mild countermeasures, strict countermeasures
R_0 = [2.5, 1.7, 0.9]
lenght_of_sickness_for_R_0 = 20 #days

#### Probabilities and intensities (following epidemiologic study)
The numbers below define the dynamic of the contagion. They are mostly based on the press releases of the CDC and on the study performed by the Vo Euganeo municipality (tested the entire population) in the epicenter of the Italian outbreak.

In [42]:

#########################
# PROBABILITY ASSUMPTIONS
#########################

asymptomatic_carriers_vs_all = 0.60 # following the study of Vo Euganeo
prob_symptomatic_from_carrier = 1 - asymptomatic_carriers_vs_all
prob_immune_from_carrier = 1 - prob_symptomatic_from_carrier
print("Share of symptomatic carriers: "+str(prob_symptomatic_from_carrier))

# using the CDC ranges to calculate the probability to need
# hospitalization once developed symptoms

w_sum_cdc = (123*sum([1.6,2.5])/2+705*sum([14.3,20.8])/2+429*sum([21.2,28.3])/2+429*sum([20.5,30.1])/2+409*sum([28.6,43.5])/2+210*sum([30.5,58.7])/2+144*sum([31.3,70.3])/2)/2449

prob_hospitalizable_from_symptomatic = w_sum_cdc/100 #following CDC estimates
print("Share of symptomatic cases needing hospitalization: "+str(prob_hospitalizable_from_symptomatic))

prob_immune_from_symptomatic = 1 - prob_hospitalizable_from_symptomatic

# using the CDC ranges to calculate the probability to die after needing
# hospitalization. The CDC calculated a death rate of 1.8-3.4% among
# all symptomatic cases
dr_cdc = ((1.8+3.4)/2)/prob_hospitalizable_from_symptomatic

prob_dead_from_hospitalizable = dr_cdc/100
print("Death rate among hospitalizable cases: "+str(prob_dead_from_hospitalizable))

prob_immune_from_hospitalizable = 1 - prob_dead_from_hospitalizable

####################
# TIME ASSUMPTIONS
####################


# avg number of days from the first contact with virus to first symptoms
carrier_to_symptomatic_avg = 6 # Source https://annals.org/aim/fullarticle/2762808/incubation-period-coronavirus-disease-2019-covid-19-from-publicly-reported

# avg number of days from carrying to become immune

carrier_to_immune_avg = 10 # Multiple sources

symptomatic_to_immune_avg = 11 # Multiple sources

symptomatic_to_hospitalizable_avg = 5 # Multiple sources

hospitalizable_to_death_avg = 4 # Source https://www.epicentro.iss.it/coronavirus/bollettino/Report-COVID-2019_17_marzo-v2.pdf

hospitalizable_to_immune_avg = 12 # Multiple sources


##########################
# TRANSITION PROBABILITIES
##########################
# the following are transition probabilities in one day

# from healthy (H) to carrier C depends on the population vector, doing it in the function

# from carrier (C) to symptomatic (S)
# computing this taking the likelihood to develop symptoms and multiplying by likelihood to get symptoms today
cs = prob_symptomatic_from_carrier * 1 / carrier_to_symptomatic_avg

# from carrier (C) to immune (I)
ci = prob_immune_from_carrier * 1 / carrier_to_immune_avg

# from symptomatic (S) to hospitalizable (X)
sx = prob_hospitalizable_from_symptomatic * 1 / symptomatic_to_hospitalizable_avg

# from symptomatic (S) to immune (I)
si = prob_immune_from_symptomatic * 1 / symptomatic_to_immune_avg

# from hospitalizable (X) to dead (D)
xd = prob_dead_from_hospitalizable * 1 / hospitalizable_to_death_avg

# from hospitalizable (X) to immune (I)
xi = prob_immune_from_hospitalizable * 1 / hospitalizable_to_immune_avg




Share of symptomatic carriers: 0.4
Share of symptomatic cases needing hospitalization: 0.267545937117
Death rate among hospitalizable cases: 0.0971795732731


### Virus transmission modeling
__H -> C__: depends on reproduction number of COVID-19, share of carriers, symptomatic, hospitalizable, healthy and immune population, and on the countermeasures implemented in the population.
One person has R_0 / (length of sickness) transmission opportunities per day to transmit/contract this virus, assuming no countermeasures. If the person is healthy, contagion happens if the transmission opportunity happens with a sick person.


In [43]:
####################
# TRANSMISSION MODEL
####################
def covid_19_one_day(population, countermeasures=0):
    H = population[state_index['H']]
    C = population[state_index['C']]
    S = population[state_index['S']]
    X = population[state_index['X']]
    D = population[state_index['D']]
    I = population[state_index['I']]

    # A person has R_0 / (length of sickness) opportunities per day to transmit/contract this virus
    # If the person is healthy, needs to meet a sick person to contract on the contract opportunity
    hc = R_0[countermeasures] / lenght_of_sickness_for_R_0 * (C + S + X)/(H + C + S + X + I)

    #Transition matrix
    covid_19 = np.array([[1-hc, hc, 0, 0, 0, 0],
                        [0, 1-cs-ci, cs, 0, 0, ci],
                        [0, 0, 1-sx-si, sx, 0, si],
                        [0, 0, 0, 1-xd-xi, xd, xi],
                        [0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 1]])
    #one step
    population_1 = np.dot(population,covid_19)
    return population_1


In [47]:
#Just a plotting function

def plot_evolution(evolution, labels = states):
    #rearrange columns
    permutation = [5,0,1,2,3,4]
    evo_for_plot = np.array(evolution)
    idx = np.empty_like(permutation)
    idx[permutation] = np.arange(len(permutation))
    evo_for_plot[:] = evo_for_plot[:, idx]  # rearrange evolution
    
    labels_for_plot = labels[:]
    for i in range(0,len(labels)):
        labels_for_plot[permutation[i]] = labels[i] #rearrange labels
    #create DataFrame for plotting
    df = pd.DataFrame(evo_for_plot, columns=labels_for_plot)
    plt = df.plot.area()
    df[["C", "X", "D"]].plot.line()
    
    
def stats_evolution(evolution, labels=states):
    final_state = evolution[len(evolution)-1]
    #Calculate the maximum number of hospitalized
    max_hospitalized = 0
    for state_day in evolution:
        if state_day[state_index['X']] > max_hospitalized:
            max_hospitalized = state_day[state_index['X']]
    
    print("Highest peak of hospitalizable people: " + str(max_hospitalized) )
    #Calculate the death rate among all carriers
    death_rate = float(final_state[state_index['D']])/(sum(final_state)-final_state[state_index['D']])
    print("Death rate: " + str(death_rate))
    
    #Show steady state
    print("Final state of the simulation: ")
    print(dict(zip(labels,final_state)))

In [55]:
def simulate_covid19(Healthy,
                     Dead, 
                     Immune, 
                     total_days,
                     social_distance_after,
                     lockdown_after):
    Healthy = float(Healthy)
    Dead = float(Dead)
    Immune = float(Immune)
    #following assumptions above
    Hospitalizable = Dead * (1 - prob_dead_from_hospitalizable) / prob_dead_from_hospitalizable
    Symptomatic = Hospitalizable * (1 - prob_hospitalizable_from_symptomatic)/prob_hospitalizable_from_symptomatic
    Carriers = Symptomatic * (1 - prob_symptomatic_from_carrier) / prob_symptomatic_from_carrier
    
    pop_vector = [Healthy, Carriers, Symptomatic, Hospitalizable, Dead , Immune]
    print("Starting population vector")
    print(pop_vector)
    all_population = sum(pop_vector)
    pop_vector = [num / all_population for num in pop_vector]
    population_vector = np.array(pop_vector)
    evolution = []

    countermeasures = 0
    for day in range(0,total_days):
        
        # updating the government countermeasures
        if day >= social_distance_after:
            countermeasures = 1
        if day >= lockdown_after:
            countermeasures = 2
            
        # simulate one day
        population_vector = covid_19_one_day(population_vector,
                                             countermeasures)
        state = [ int(num*all_population) for num in population_vector]
        evolution.append(state)

    evolution = np.asarray(evolution)
    plot_evolution(evolution)
    stats_evolution(evolution)
    

### Simulation
Specify the population of the country and the number of deaths for COVID-19 to simulate the outbreak. Specify how long it takes the government to **implement social distance** or to **enforce a lockdown** (in number of days)

In [56]:
@interact
def simulate_interactive(Population='61000000',
                         Dead='100', 
                         Immune='0',
                         social_distance = (0,600,10),
                         lockdown = (20,600,10),
                         simulation_days = 300):
    simulate_covid19(Population,
                     Dead,
                     Immune,
                     simulation_days,
                     social_distance,
                     lockdown)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFRleHQodmFsdWU9dSc2MTAwMDAwMCcsIGRlc2NyaXB0aW9uPXUnUG9wdWxhdGlvbicpLCBUZXh0KHZhbHVlPXUnMTAwJywgZGVzY3JpcHRpb249dSdEZWHigKY=


9.71795732731
